In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from scipy.stats import norm

from tqdm import tqdm

# Data Loading

In [ ]:
BIKE_COUNT_URLS_AND_LOAD_KWARGS = [
    ("https://at.govt.nz/media/1991027/auckland-trasnport-cycle-counts-jan-dec-2022.xlsx", {}),
    ("https://at.govt.nz/media/1991376/auckland-transport-january-2023-cycle-counts.xlsx", {}),
    ("https://at.govt.nz/media/1991377/auckland-transport-february-2023-cycle-counts.xlsx", {}),
    ("https://at.govt.nz/media/1991436/auckland-transport-march-2023-cycle-counts.xlsx", {"na_values": "Pending"}),
    ("https://at.govt.nz/media/1992545/april-2023-cycle-counts.xlsx", {}),
    ("https://at.govt.nz/media/1992547/may-2023-cycle-counts.xlsx", {}),
    ("https://at.govt.nz/media/1992546/june-2023-cycle-counts.xlsx", {}),
    ("https://at.govt.nz/media/xs5lybun/at-daily-cycle-data-july-2023.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/t3zlbdn4/at-daily-cycle-data-august-2023.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/53rj40ji/at-daily-cycle-data-september-2023.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/hl0jbsrl/at-daily-cycle-data-october-2023.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/ty4js45t/at-daily-cycle-counts-november-2023.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/1tvdgsgo/at-daily-cycle-counts-december-2023.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/bb4h3wd3/at-daily-cycle-counts-january-2024.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/xlcaru0v/at-daily-cycle-counts-feb-2024.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/4g3hzpp5/at-daily-cycle-counts-march-2024.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/htvezqdn/at-daily-cycle-counts-april-2024.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/ue5cygl0/at-daily-cycle-counts-may-2024.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/3icd2jug/at-daily-cycle-counts-june-2024.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/jbdd1rox/cycle-counts-july-2024.xlsx", {"header": 2, "na_values": "z"}),
    ("https://at.govt.nz/media/bvadzmqg/cycle-counts-august-2024.xlsx", {"usecols": "G:CG"}),
    ("https://at.govt.nz/media/lpsfdwbe/auckland-transport-cycle-counts-september-2024.xlsx", {"usecols": "G:CG"}),
    ("https://at.govt.nz/media/ohbhvmrl/auckland-transport-cycle-movements-october-2024.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/chelni1f/auckland-transport-cycle-movements-data-november-2024.xlsx", {"header": 2}),
    ("https://at.govt.nz/media/zdumuud2/auckland-transport-cycle-movements-december-2024.xlsx", {"header": 2}),
]

In [ ]:
LOCATION_MAPPING = {
    "GI TO TAMAKI DR SECTION-1": "GI to Tamaki Drive Section-1",
    "Great North Road NB Towards CBD": "Great North Road",
    "Great South Road Manukau": "Great South Road",
    "Highbrook Pathway": "Highbrook Drive",
    "Lake Road Total New": "Lake Road",
    "Leigh Road Shared Path": "Leigh Road",
    "Mangere Foot Bridge": "Mangere Bridge",
    "Ocean View Road": "Oceanview Road",
    "Rathgar Road": "Rathger Road",
    "Remuera Road": "Remurua Road",
    "SH18 Upper Harbour Drive": "Upper Harbour",
    "SH20A Shared Path": "SH20A",
    "SW Shared Path": "SW SH20",
    "Symonds Street Total": "Symonds Street",
    "Tamaki Drive - Parnell": "Tamaki Drive",
    "TeAtatu Peninsula Shared Path": "TeAtatu Peninsula",
    "TeWero Bridge Bike Counter": "TeWero Bridge",
    "Ti Rakau - Opposite Bunnings": "Ti Rakau",
    "Ti Rakau opposite 92": "Ti Rakau",
    "Twin Streams Shared Path": "Twin Streams",
    "Upper Harbour Shared Path": "Upper Harbour",
    "Waterview Unitec Shared Path": "Waterview Unitec",
}

LOCATIONS_TO_DROP = [
    "Ngapipi Road SUP",
    "Meadowbank SUP",
    "Rankin Ave Shared Path",
    "Ti Rakau",
    "Ti Rakau Reserve",
    "Ti Rakau Riverhills",
    "TeWero Bridge",
]

In [ ]:
def load_cycle_counts(url: str, **read_excel_kwargs) -> pd.DataFrame:
    df = pd.read_excel(url, **read_excel_kwargs)
    df = df.rename(columns={"Date": "date", "Date.1": "date", "Time": "date"})
    df = df.melt(id_vars="date", var_name="location", value_name="count")
    df = df.astype({"date": "datetime64[ns]", "location": str, "count": float})
    return df


def map_locations(location: str) -> str:
    return LOCATION_MAPPING.get(location, location)


def standardise_locations(df: pd.DataFrame) -> pd.DataFrame:
    df["location"] = df["location"].str.replace(r" Cyclists?", "", regex=True)
    df["location"] = df["location"].str.replace(r"\bDr\b", "Drive", regex=True)
    df["location"] = df["location"].str.replace(r"\bRd\b", "Road", regex=True)
    df["location"] = df["location"].str.replace(r"\bSt\b", "Street", regex=True)
    df["location"] = df["location"].apply(map_locations)
    return df


def drop_locations(df: pd.DataFrame) -> pd.DataFrame:
    to_drop = df["location"].isin(LOCATIONS_TO_DROP)
    return df.loc[~to_drop].copy()

In [ ]:
dfs = []
for url, load_kwargs in tqdm(BIKE_COUNT_URLS_AND_LOAD_KWARGS):
    try:
        df = load_cycle_counts(url, **load_kwargs)
    except Exception as e:
        print("Could not load data for url: ", url)
        print("Reason: ", e)
        continue
    dfs.append(df)

cycle_counts = pd.concat(dfs, axis=0, ignore_index=True)
cycle_counts = standardise_locations(cycle_counts)
cycle_counts = drop_locations(cycle_counts)

In [ ]:
i = 0
locs = sorted(cycle_counts["location"].unique())[i * 10:  (i + 1) * 10]
print(locs)

fig, ax = plt.subplots(len(locs), 1, figsize=(10, len(locs) * 1.75), sharex=True)
for i, loc in enumerate(locs):
    outlier_df = cycle_counts[cycle_counts["location"] == loc].sort_values("date")
    ax[i].plot(
        outlier_df["date"].values,
        outlier_df["count"].values,
        label=loc
    )
    ax[i].legend()
    # ax[i].set_ylim(0, 500)

fig.tight_layout();

In [ ]:
# Outlier
outlier_locs = [
    "Albany Highway",
    "Archibald Park",
    "GI to Tamaki Drive Section-1",
    "Lake Road",
    "Mangere Safe Routes",
    "NW Cycleway TeAtatu",
    "Sandringham Road",
    "SH20A",
    "SW SH20",
]

In [ ]:
def get_outlier_df(location_df: pd.DataFrame, n_sigma: float = 3.0) -> pd.DataFrame:
    outlier_df = location_df.copy()

    # Compute mean and std by weekday
    outlier_df = outlier_df.assign(weekday=outlier_df["date"].dt.weekday)
    weekday_stats = (
        outlier_df
        .groupby("weekday")[["count"]]
        .agg(["mean", "std"])
        .droplevel(level=0, axis=1)
        .reset_index()
    )
    outlier_df = outlier_df.merge(weekday_stats, how="left", on="weekday")
    
    # Define upper and lower bounds
    outlier_df = outlier_df.assign(
        upper=outlier_df["mean"] + n_sigma * outlier_df["std"],
        lower=outlier_df["mean"] - n_sigma * outlier_df["std"],
    )
    is_outlier = (outlier_df["count"] > outlier_df["upper"]) | (outlier_df["count"] < outlier_df["lower"])
    outlier_df = outlier_df.assign(is_outlier=is_outlier)
    
    return outlier_df

In [ ]:
loc = outlier_locs[9]
location_df = cycle_counts[cycle_counts["location"] == loc]

n_sigma = 3.0
n_expected = len(location_df) * 2 * norm().cdf(-n_sigma)

outlier_df = get_outlier_df(location_df, n_sigma=n_sigma)
outlier_df = outlier_df.sort_values("date")
fig, ax = plt.subplots(figsize=(10, 3.5))
ax.plot(outlier_df["date"].values, outlier_df["count"].values)
ax.plot(outlier_df["date"].values, outlier_df["upper"].values, ls="--")
ax.plot(outlier_df["date"].values, outlier_df["lower"].values, ls="--")
ax.set(title=loc)

In [ ]:
# Adjust level
locs = [
    "Great South Road",
    "Karangahape Road",
]

In [ ]:
cycle_counts.to_csv("cycle_counts.csv", index=False)

In [ ]:
cycle_counts = pd.read_csv("cycle_counts.csv", parse_dates=["time"])

## EDA

### Total counts by day

In [ ]:
daily_total = cycle_counts.groupby("date")[["count"]].sum().reset_index()

fig, ax = plt.subplots(1, 2, figsize=(10, 3.5))

ax[0].plot(daily_total["date"], daily_total["count"])
ax[0].set(ylabel="Count")
for tick in ax[0].get_xticklabels():
    tick.set_rotation(45)

ax[1].hist(daily_total["count"], bins=20);
ax[1].set(xlabel="Count", ylabel="Frequency")

fig.tight_layout()

### Daily counts by location

In [ ]:
av_daily_counts = cycle_counts.groupby("date")[["count"]].mean().reset_index()

fig, ax = plt.subplots(2, 1, figsize=(12, 5), sharex=True)

locations = cycle_counts["location"].unique()
for location in locations:
    outlier_df = cycle_counts[cycle_counts["location"] == location]
    ax[0].plot(outlier_df["date"], outlier_df["count"], color="gray", alpha=0.5)
    ax[1].plot(outlier_df["date"], np.log1p(outlier_df["count"]), color="gray", alpha=0.5)

ax[0].plot(av_daily_counts["date"], av_daily_counts["count"], color="blue", linewidth=2)
ax[1].plot(av_daily_counts["date"], np.log1p(av_daily_counts["count"]), color="blue", linewidth=2)

ax[0].set(ylabel="Count")
ax[1].set(ylabel="Log(Count)")

In [ ]:
av_daily_counts_by_loc = cycle_counts.groupby("location")[["count"]].mean().reset_index()
av_daily_counts_by_loc = av_daily_counts_by_loc.sort_values("count").reset_index(drop=True)

mean_counts = av_daily_counts_by_loc["count"].mean()
median_counts = av_daily_counts_by_loc["count"].median()

fig, ax = plt.subplots(1, 1)
ax.hist(av_daily_counts_by_loc["count"], bins=20)
ax.axvline(mean_counts, color="red", linestyle="--", lw=2.5, label="Mean")
ax.axvline(median_counts, color="orange", linestyle="--", lw=2.5, label="Median")
ax.set(xlabel="Average Daily Count", ylabel="Frequency")
ax.legend();

### High demand locations

In [ ]:
threshold = 600
high_demand_locs = av_daily_counts_by_loc[av_daily_counts_by_loc["count"] > threshold]
high_demand_locs.sort_values("count")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

locations = high_demand_locs["location"].values
colors = cm.coolwarm(np.linspace(0, 1, len(locations)))
for i, location in enumerate(locations):
    outlier_df = cycle_counts[cycle_counts["location"] == location]
    ax.plot(
        outlier_df["date"],
        outlier_df["count"],
        label=location,
        color=colors[i],
        lw=2.5,
    )
ax.set(ylabel="Count")
fig.tight_layout();

In [ ]:
counts_pivot = cycle_counts.pivot_table(columns="date", index="location", values="count", aggfunc="sum")
counts_pivot = counts_pivot.fillna(0).assign(average=lambda x: x.mean(axis=1))
counts_pivot = counts_pivot[counts_pivot["average"].between(100, 200)]
counts_pivot = counts_pivot.sort_values("average", ascending=False).drop(columns="average")

fig, ax = plt.subplots(1, 1)
sns.heatmap(counts_pivot, cmap="coolwarm", cbar_kws={"label": "Count"})

xs = np.arange(0, counts_pivot.shape[1], 60)
xticklables = [c.strftime("%Y-%m-%d") for c in counts_pivot.columns[xs]]
ax.set(xticks=xs, xticklabels=xticklables)

ys = np.arange(0, counts_pivot.shape[0])
yticklables = counts_pivot.index[ys]
ax.set_yticks(ys + 0.5)
ax.set_yticklabels(yticklables, fontsize="small")

ax.set(xlabel="", ylabel="");